In [14]:
import numpy as np
import sys
import ipdb

sys.path.append("../")

sys.path.append("../utils/")
from utils_functions import *

from dd_game import *
from two_stage_player import *

# Configurations

In [15]:
#TODO: There are many configuration parameters here. It is important to understand the effect of all.
d = 2
coeff_var = 1

Sigma_x_p1 = np.eye(d)
Sigma_x_p2 = np.eye(d)

sigma_y_p1 = 1
sigma_y_p2 = 1

beta_p1 = np.random.normal(size = d)
beta_p2 = np.random.normal(size = d)

"""
mu_p1 = np.random.normal(scale = coeff_var, size = d)
mu_p2 = np.random.normal(scale = coeff_var, size = d)
gamma_p1 = np.random.normal(scale = coeff_var, size = d)
gamma_p2 = np.random.normal(scale = coeff_var, size = d)
"""

#Fixing mu and gamma for testing purposes
mu_p1 = np.array([2.1, -1.8])
gamma_p1 = np.array([1.2, 0.9])
mu_p2 = np.array([-2.8, 2.2])
gamma_p2 = np.array([0.7, -0.3])

In [16]:
p1_data_params = (Sigma_x_p1, sigma_y_p1, beta_p1, mu_p1, gamma_p1)
p1_data_generating_func = sample_from_location_family

p2_data_params = (Sigma_x_p2, sigma_y_p2, beta_p2, mu_p2, gamma_p2)
p2_data_generating_func = sample_from_location_family

num_rounds_lst = [10, 100, 1000, 10000, 40000]
num_alternate_rounds = 1000
num_test = 10000

# Run Game

In [17]:
#Initialize game with configurations specified above


for num_rounds in num_rounds_lst:
    player_two = TwoStagePlayer()
    player_one = TwoStagePlayer()
    game = DecisionDependentGame(player_one, player_two, p1_data_params, p2_data_params, 
                             p1_data_generating_func, p2_data_generating_func, 
                             num_rounds, num_alternate_rounds, num_test)
    game.run_train()
    print("---------------------")
    print("Player 1")
    print(mu_p1, gamma_p1)
    print(player_one.mu_hat, player_one.gamma_hat)
    
    print("Player 2")
    print(mu_p2, gamma_p2)
    print(player_two.mu_hat, player_two.gamma_hat)

Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
---------------------
Player 1
[ 2.1 -1.8] [1.2 0.9]
[ 2.59658323 -1.53613243] [ 0.04367939 -2.22639563]
Player 2
[-2.8  2.2] [ 0.7 -0.3]
[-3.23211196 -0.13386714] [ 0.6474658  -1.50188569]
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
---------------------
Player 1
[ 2.1 -1.8] [1.2 0.9]
[ 2.02904738 -1.94941939] [1.63571681 0.63913216]
Player 2
[-2.8  2.2] [ 0.7 -0.3]
[-2.66656638  2.19608912] [ 0.73466731 -0.48520135]
Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.
---------------------
Player 1
[ 2.1 -1.8] [1.2 0.9]
[ 2.12504064 -1.73713103] [1.13866399 0.82581274]
Player 2
[-2.8  2.2] [ 0.7 -0.3]
[-2.74940494  2.19

In [5]:
#game.run_train()

## Checking stage 1

In [6]:
print(player_one.mu_hat, player_one.gamma_hat)
print(player_two.mu_hat, player_two.gamma_hat)

[ 2.19722211 -1.65796154] [1.14222762 0.91727065]
[-3.07443107  2.25700635] [ 0.52519599 -0.42916865]


In [7]:
print(mu_p1, gamma_p1)
print(mu_p2, gamma_p2)

[ 2.1 -1.8] [1.2 0.9]
[-2.8  2.2] [ 0.7 -0.3]


## Evaluate the performative risks before and after Stage 3

In [8]:
game.evaluate_test_perf_risk_p1()

124.5059923748503

In [9]:
game.evaluate_test_perf_risk_p2()

103.75116427300328

In [10]:
player_one.theta_history[-1]
player_two.theta_history[-1]

array([-0.10587816, -0.63755366])